In [113]:
from __future__ import division
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
import sklearn as sk
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
import warnings # to silence convergence warnings
import scipy.linalg as lng 
# seaborn can be used to "prettify" default matplotlib plots by importing and setting as default
import seaborn as sns
sns.set() # Set searborn as default
from sklearn.preprocessing import StandardScaler

# Load data

In [114]:
case_01 = pd.read_csv('../case1Data.txt', sep = ",", engine='python')
case_01

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,3.692902,12.826477,-6.889479,3.443535,5.400120,I,NaN,H,J,K
1,-22.200026,-6.370690,7.792366,11.858663,NaN,-15.420530,21.683970,1.057192,2.559764,NaN,...,7.270303,10.039281,-7.678936,-0.189822,0.717526,H,NaN,H,I,I
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,NaN,2.484515,NaN,1.852533,...,5.889789,14.186614,-9.527944,0.400832,-2.458897,NaN,I,K,J,G
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,NaN,-0.442587,0.743039,-0.376102,...,7.024105,8.871258,-7.514400,-3.448892,-2.612379,K,H,I,I,NaN
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,NaN,19.057921,1.900578,6.133609,-0.771247,...,8.186575,10.532577,-7.509476,NaN,0.754905,H,H,K,J,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,NaN,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,NaN,17.348757,-6.059881,4.196626,1.168894,H,I,G,NaN,H
96,-0.623049,-4.311540,7.208115,NaN,-17.232457,-11.173426,24.660276,NaN,8.513152,4.327020,...,9.753088,17.345148,-3.650362,3.535945,NaN,H,I,I,J,K
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,NaN,...,-0.300405,7.651474,NaN,-4.384652,-1.665588,G,H,NaN,G,K
98,-25.058153,-7.338365,1.597154,12.151154,NaN,NaN,23.185303,3.751106,6.357028,5.208797,...,6.465018,11.647115,-4.830073,-2.378979,-3.120303,G,I,I,H,NaN


In [115]:
case_01_new = pd.read_csv('../case1Data_XNew.txt',sep=',',engine='python')
case_01_new

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.493200,4.857913,12.334877,-17.075041,NaN,NaN,5.339252,NaN,-0.275014,NaN,...,6.983981,14.086256,NaN,3.391461,2.102585,G,I,G,J,NaN
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,5.799896,11.477398,-10.509565,-0.347968,-2.471949,I,I,NaN,NaN,H
2,-8.507726,1.639164,NaN,-19.167274,-15.745444,20.336708,NaN,NaN,2.756582,-6.619169,...,9.498865,NaN,-6.555100,-3.588808,-2.342992,NaN,H,NaN,H,K
3,-12.841493,NaN,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,8.428060,12.257391,-8.267102,-1.373772,-5.374916,G,H,K,K,J
4,-8.555160,NaN,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,NaN,...,11.065901,8.004617,-10.170813,2.445537,NaN,H,I,J,H,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,NaN,3.956915,-1.673680,-5.303247,...,8.286523,14.119678,-6.228438,-0.419140,-4.737494,NaN,I,H,I,NaN
996,-7.835061,5.410653,NaN,-18.169995,-15.071277,23.569444,2.922559,5.423952,NaN,-0.561963,...,8.773873,NaN,-8.891943,0.617816,NaN,J,I,NaN,G,G
997,-7.547603,NaN,NaN,NaN,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,NaN,13.666244,-9.850952,-2.478082,1.597624,H,H,H,H,J
998,-10.989069,3.416966,NaN,-17.205503,-10.092004,21.299033,2.201629,3.492949,NaN,-4.334677,...,12.008307,12.634788,-5.447805,-0.529636,1.834458,H,I,G,G,G


## We tried converting string "NaN" to np.nan and drop the rows which have them, but both of files have NaN in every rows. (Do not run this)

In [62]:
case_01_data = case_01.copy()
case_01_data_new = case_01_new.copy()

In [63]:
# df = df.replace(0, np.NaN)
case_01_data = case_01_data.replace(" NaN",np.nan)
case_01_data = case_01_data.replace("NaN ",np.nan)
case_01_data = case_01_data.replace("NaN",np.nan)

case_01_data_new = case_01_data_new.replace(" NaN",np.nan)
case_01_data_new = case_01_data_new.replace("NaN",np.nan)
case_01_data_new = case_01_data_new.replace("NaN ",np.nan)

In [64]:
case_01_data.isnull().sum(1) 

0     15
1     15
2     14
3     10
4     15
      ..
95    14
96    15
97    19
98    23
99    21
Length: 100, dtype: int64

In [65]:
case_01_cp = case_01_data.copy()
case_01_new_cp = case_01_data_new.copy()

In [66]:
case_01_cp

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,3.692902,12.826477,-6.889479,3.443535,5.400120,I,NaN,H,J,K
1,-22.200026,-6.370690,7.792366,11.858663,NaN,-15.420530,21.683970,1.057192,2.559764,NaN,...,7.270303,10.039281,-7.678936,-0.189822,0.717526,H,NaN,H,I,I
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,NaN,2.484515,NaN,1.852533,...,5.889789,14.186614,-9.527944,0.400832,-2.458897,NaN,I,K,J,G
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,NaN,-0.442587,0.743039,-0.376102,...,7.024105,8.871258,-7.514400,-3.448892,-2.612379,K,H,I,I,NaN
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,NaN,19.057921,1.900578,6.133609,-0.771247,...,8.186575,10.532577,-7.509476,NaN,0.754905,H,H,K,J,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,NaN,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,NaN,17.348757,-6.059881,4.196626,1.168894,H,I,G,NaN,H
96,-0.623049,-4.311540,7.208115,NaN,-17.232457,-11.173426,24.660276,NaN,8.513152,4.327020,...,9.753088,17.345148,-3.650362,3.535945,NaN,H,I,I,J,K
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,NaN,...,-0.300405,7.651474,NaN,-4.384652,-1.665588,G,H,NaN,G,K
98,-25.058153,-7.338365,1.597154,12.151154,NaN,NaN,23.185303,3.751106,6.357028,5.208797,...,6.465018,11.647115,-4.830073,-2.378979,-3.120303,G,I,I,H,NaN


In [67]:
case_01_new_cp

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.493200,4.857913,12.334877,-17.075041,NaN,NaN,5.339252,NaN,-0.275014,NaN,...,6.983981,14.086256,NaN,3.391461,2.102585,G,I,G,J,NaN
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,5.799896,11.477398,-10.509565,-0.347968,-2.471949,I,I,NaN,NaN,H
2,-8.507726,1.639164,NaN,-19.167274,-15.745444,20.336708,NaN,NaN,2.756582,-6.619169,...,9.498865,NaN,-6.555100,-3.588808,-2.342992,NaN,H,NaN,H,K
3,-12.841493,NaN,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,8.428060,12.257391,-8.267102,-1.373772,-5.374916,G,H,K,K,J
4,-8.555160,NaN,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,NaN,...,11.065901,8.004617,-10.170813,2.445537,NaN,H,I,J,H,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,NaN,3.956915,-1.673680,-5.303247,...,8.286523,14.119678,-6.228438,-0.419140,-4.737494,NaN,I,H,I,NaN
996,-7.835061,5.410653,NaN,-18.169995,-15.071277,23.569444,2.922559,5.423952,NaN,-0.561963,...,8.773873,NaN,-8.891943,0.617816,NaN,J,I,NaN,G,G
997,-7.547603,NaN,NaN,NaN,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,NaN,13.666244,-9.850952,-2.478082,1.597624,H,H,H,H,J
998,-10.989069,3.416966,NaN,-17.205503,-10.092004,21.299033,2.201629,3.492949,NaN,-4.334677,...,12.008307,12.634788,-5.447805,-0.529636,1.834458,H,I,G,G,G


### Drop the row which has NaN

In [68]:
print("case_01 shape:{}".format(case_01_cp.shape))
print("case_01_new shape:{}".format(case_01_new_cp.shape))

case_01 shape:(100, 101)
case_01_new shape:(1000, 100)


In [69]:
# df.dropna(axis=0) -> for dropping row which has nan
case_01_cp = case_01_cp.dropna(axis=0)
case_01_new_cp = case_01_new_cp.dropna(axis=0)

print("After dropping rows which have NaN")
print("case_01 shape:{}".format(case_01_cp.shape)) # all the rows in case1Data.txt have at least 1 NaN
print("case_01_new shape:{}".format(case_01_new_cp.shape)) # all the rows in case1Data_XNew.txt have at least 1 NaN

After dropping rows which have NaN
case_01 shape:(0, 101)
case_01_new shape:(0, 100)


In [70]:
case_01_cp = case_01_cp.dropna(axis=0)
case_01_cp

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5


In [71]:
case_01_new_cp

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5


## So, firstly, we converted string nan to np.nan, then converted np.nan to the average value of column that data is belong to (Do not run this)
-> It didn't work because there are columns which have NaN, so they cannot make mean.

In [4]:
case1Data = case_01.copy()
case1Data_XNew = case_01_new.copy()

In [5]:
case1Data

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,3.692902,12.826477,-6.889479,3.443535,5.400120,I,NaN,H,J,K
1,-22.200026,-6.370690,7.792366,11.858663,NaN,-15.420530,21.683970,1.057192,2.559764,NaN,...,7.270303,10.039281,-7.678936,-0.189822,0.717526,H,NaN,H,I,I
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,NaN,2.484515,NaN,1.852533,...,5.889789,14.186614,-9.527944,0.400832,-2.458897,NaN,I,K,J,G
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,NaN,-0.442587,0.743039,-0.376102,...,7.024105,8.871258,-7.514400,-3.448892,-2.612379,K,H,I,I,NaN
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,NaN,19.057921,1.900578,6.133609,-0.771247,...,8.186575,10.532577,-7.509476,NaN,0.754905,H,H,K,J,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,NaN,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,NaN,17.348757,-6.059881,4.196626,1.168894,H,I,G,NaN,H
96,-0.623049,-4.311540,7.208115,NaN,-17.232457,-11.173426,24.660276,NaN,8.513152,4.327020,...,9.753088,17.345148,-3.650362,3.535945,NaN,H,I,I,J,K
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,NaN,...,-0.300405,7.651474,NaN,-4.384652,-1.665588,G,H,NaN,G,K
98,-25.058153,-7.338365,1.597154,12.151154,NaN,NaN,23.185303,3.751106,6.357028,5.208797,...,6.465018,11.647115,-4.830073,-2.378979,-3.120303,G,I,I,H,NaN


In [6]:
case1Data_XNew

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.493200,4.857913,12.334877,-17.075041,NaN,NaN,5.339252,NaN,-0.275014,NaN,...,6.983981,14.086256,NaN,3.391461,2.102585,G,I,G,J,NaN
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,5.799896,11.477398,-10.509565,-0.347968,-2.471949,I,I,NaN,NaN,H
2,-8.507726,1.639164,NaN,-19.167274,-15.745444,20.336708,NaN,NaN,2.756582,-6.619169,...,9.498865,NaN,-6.555100,-3.588808,-2.342992,NaN,H,NaN,H,K
3,-12.841493,NaN,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,8.428060,12.257391,-8.267102,-1.373772,-5.374916,G,H,K,K,J
4,-8.555160,NaN,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,NaN,...,11.065901,8.004617,-10.170813,2.445537,NaN,H,I,J,H,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,NaN,3.956915,-1.673680,-5.303247,...,8.286523,14.119678,-6.228438,-0.419140,-4.737494,NaN,I,H,I,NaN
996,-7.835061,5.410653,NaN,-18.169995,-15.071277,23.569444,2.922559,5.423952,NaN,-0.561963,...,8.773873,NaN,-8.891943,0.617816,NaN,J,I,NaN,G,G
997,-7.547603,NaN,NaN,NaN,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,NaN,13.666244,-9.850952,-2.478082,1.597624,H,H,H,H,J
998,-10.989069,3.416966,NaN,-17.205503,-10.092004,21.299033,2.201629,3.492949,NaN,-4.334677,...,12.008307,12.634788,-5.447805,-0.529636,1.834458,H,I,G,G,G


In [7]:
case1Data = case1Data.replace(" NaN",np.nan)
case1Data = case1Data.replace("NaN ",np.nan)
case1Data = case1Data.replace("NaN",np.nan)
case1Data = case1Data.fillna(case1Data.mean())

case1Data_XNew = case1Data_XNew.replace(" NaN",np.nan)
case1Data_XNew = case1Data_XNew.replace("NaN",np.nan)
case1Data_XNew = case1Data_XNew.replace("NaN ",np.nan)
case1Data_XNew = case1Data_XNew.fillna(case1Data_XNew.mean())

C:\Users\user\AppData\Local\Temp\ipykernel_5036\2600407623.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  case1Data = case1Data.fillna(case1Data.mean())
C:\Users\user\AppData\Local\Temp\ipykernel_5036\2600407623.py:9: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  case1Data_XNew = case1Data_XNew.fillna(case1Data_XNew.mean())


In [8]:
case1Data

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,3.692902,12.826477,-6.889479,3.443535,5.400120,I,NaN,H,J,K
1,-22.200026,-6.370690,7.792366,11.858663,NaN,-15.420530,21.683970,1.057192,2.559764,NaN,...,7.270303,10.039281,-7.678936,-0.189822,0.717526,H,NaN,H,I,I
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,NaN,2.484515,NaN,1.852533,...,5.889789,14.186614,-9.527944,0.400832,-2.458897,NaN,I,K,J,G
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,NaN,-0.442587,0.743039,-0.376102,...,7.024105,8.871258,-7.514400,-3.448892,-2.612379,K,H,I,I,NaN
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,NaN,19.057921,1.900578,6.133609,-0.771247,...,8.186575,10.532577,-7.509476,NaN,0.754905,H,H,K,J,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,NaN,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,NaN,17.348757,-6.059881,4.196626,1.168894,H,I,G,NaN,H
96,-0.623049,-4.311540,7.208115,NaN,-17.232457,-11.173426,24.660276,NaN,8.513152,4.327020,...,9.753088,17.345148,-3.650362,3.535945,NaN,H,I,I,J,K
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,NaN,...,-0.300405,7.651474,NaN,-4.384652,-1.665588,G,H,NaN,G,K
98,-25.058153,-7.338365,1.597154,12.151154,NaN,NaN,23.185303,3.751106,6.357028,5.208797,...,6.465018,11.647115,-4.830073,-2.378979,-3.120303,G,I,I,H,NaN


In [9]:
case1Data_XNew

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,-6.493200,4.857913,12.334877,-17.075041,NaN,NaN,5.339252,NaN,-0.275014,NaN,...,6.983981,14.086256,NaN,3.391461,2.102585,G,I,G,J,NaN
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,5.799896,11.477398,-10.509565,-0.347968,-2.471949,I,I,NaN,NaN,H
2,-8.507726,1.639164,NaN,-19.167274,-15.745444,20.336708,NaN,NaN,2.756582,-6.619169,...,9.498865,NaN,-6.555100,-3.588808,-2.342992,NaN,H,NaN,H,K
3,-12.841493,NaN,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,8.428060,12.257391,-8.267102,-1.373772,-5.374916,G,H,K,K,J
4,-8.555160,NaN,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,NaN,...,11.065901,8.004617,-10.170813,2.445537,NaN,H,I,J,H,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,NaN,3.956915,-1.673680,-5.303247,...,8.286523,14.119678,-6.228438,-0.419140,-4.737494,NaN,I,H,I,NaN
996,-7.835061,5.410653,NaN,-18.169995,-15.071277,23.569444,2.922559,5.423952,NaN,-0.561963,...,8.773873,NaN,-8.891943,0.617816,NaN,J,I,NaN,G,G
997,-7.547603,NaN,NaN,NaN,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,NaN,13.666244,-9.850952,-2.478082,1.597624,H,H,H,H,J
998,-10.989069,3.416966,NaN,-17.205503,-10.092004,21.299033,2.201629,3.492949,NaN,-4.334677,...,12.008307,12.634788,-5.447805,-0.529636,1.834458,H,I,G,G,G


In [ ]:
for i in range(100):
    for j in range(101):
        if "NaN" in case1Data.iloc[i][j] or case1Data.iloc[i][j]==np.nan:
            # case1Data.iloc[i][j] = '0.0'
            case1Data.loc[i,case1Data.columns[j]] = '0.0'
case1Data

## Method3 -> Converting both string "NaN" and np.nan to 0.0 (Run this!)
### In the case of first column in the x new dataset, the mode value of the first column is NaN, so all the NaN in the first column could not be changed.
### We are going to drop the rows after all process

In [116]:
case_01_numerical = case_01[case_01.columns[:96]]
case_01_categorical = case_01[case_01.columns[96:]]

In [117]:
case_01_numerical

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,-2.590210,-1.943897,3.011972,-31.614019,-3.990017,3.692902,12.826477,-6.889479,3.443535,5.400120
1,-22.200026,-6.370690,7.792366,11.858663,NaN,-15.420530,21.683970,1.057192,2.559764,NaN,...,-3.573072,-1.583166,NaN,-28.880360,-6.552140,7.270303,10.039281,-7.678936,-0.189822,0.717526
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,NaN,2.484515,NaN,1.852533,...,-4.023667,2.648645,4.067629,-32.769482,-5.438118,5.889789,14.186614,-9.527944,0.400832,-2.458897
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,NaN,-0.442587,0.743039,-0.376102,...,-4.987491,3.992532,3.954274,-35.846878,-10.208559,7.024105,8.871258,-7.514400,-3.448892,-2.612379
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,NaN,19.057921,1.900578,6.133609,-0.771247,...,-4.021144,3.152862,3.366474,NaN,-3.632849,8.186575,10.532577,-7.509476,NaN,0.754905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,NaN,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,-2.473629,3.219622,7.296911,-31.600305,-1.008369,NaN,17.348757,-6.059881,4.196626,1.168894
96,-0.623049,-4.311540,7.208115,NaN,-17.232457,-11.173426,24.660276,NaN,8.513152,4.327020,...,1.541280,6.440714,4.503169,-29.506432,-6.276006,9.753088,17.345148,-3.650362,3.535945,NaN
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,NaN,...,-11.148246,NaN,-2.396112,-39.440232,-8.566806,-0.300405,7.651474,NaN,-4.384652,-1.665588
98,-25.058153,-7.338365,1.597154,12.151154,NaN,NaN,23.185303,3.751106,6.357028,5.208797,...,-6.120490,NaN,0.672467,-33.044920,-5.693242,6.465018,11.647115,-4.830073,-2.378979,-3.120303


In [118]:
case_01_categorical

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,I,NaN,H,J,K
1,H,NaN,H,I,I
2,NaN,I,K,J,G
3,K,H,I,I,NaN
4,H,H,K,J,J
...,...,...,...,...,...
95,H,I,G,NaN,H
96,H,I,I,J,K
97,G,H,NaN,G,K
98,G,I,I,H,NaN


In [119]:
case_01_new_numerical = case_01_new[case_01_new.columns[:95]]
case_01_new_categorical = case_01_new[case_01_new.columns[95:]]

In [120]:
case_01_new_numerical

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.493200,4.857913,12.334877,-17.075041,NaN,NaN,5.339252,NaN,-0.275014,NaN,...,-2.661989,3.802699,6.012018,-30.831163,NaN,6.983981,14.086256,NaN,3.391461,2.102585
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,-4.195056,2.549679,4.436252,NaN,-6.435776,5.799896,11.477398,-10.509565,-0.347968,-2.471949
2,-8.507726,1.639164,NaN,-19.167274,-15.745444,20.336708,NaN,NaN,2.756582,-6.619169,...,-3.809937,2.671304,3.119847,-33.272728,-3.679832,9.498865,NaN,-6.555100,-3.588808,-2.342992
3,-12.841493,NaN,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,-8.390266,0.930408,2.797268,-36.442482,-7.670354,8.428060,12.257391,-8.267102,-1.373772,-5.374916
4,-8.555160,NaN,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,NaN,...,-1.446679,NaN,7.196290,-31.796824,-7.212525,11.065901,8.004617,-10.170813,2.445537,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,NaN,3.956915,-1.673680,-5.303247,...,-6.531539,-1.911204,4.902708,-32.978883,-7.900873,8.286523,14.119678,-6.228438,-0.419140,-4.737494
996,-7.835061,5.410653,NaN,-18.169995,-15.071277,23.569444,2.922559,5.423952,NaN,-0.561963,...,-2.772379,3.177223,3.005404,-30.616184,-4.494307,8.773873,NaN,-8.891943,0.617816,NaN
997,-7.547603,NaN,NaN,NaN,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,-5.460523,0.778803,4.249185,-32.984152,-3.465181,NaN,13.666244,-9.850952,-2.478082,1.597624
998,-10.989069,3.416966,NaN,-17.205503,-10.092004,21.299033,2.201629,3.492949,NaN,-4.334677,...,-1.871098,NaN,3.734328,-33.292990,NaN,12.008307,12.634788,-5.447805,-0.529636,1.834458


In [121]:
case_01_new_categorical

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,G,I,G,J,NaN
1,I,I,NaN,NaN,H
2,NaN,H,NaN,H,K
3,G,H,K,K,J
4,H,I,J,H,G
...,...,...,...,...,...
995,NaN,I,H,I,NaN
996,J,I,NaN,G,G
997,H,H,H,H,J
998,H,I,G,G,G


In [122]:
case_01_numerical_cp = case_01_numerical.copy()
case_01_categorical_cp = case_01_categorical.copy()

case_01_new_numerical_cp = case_01_new_numerical.copy()
case_01_new_categorical_cp = case_01_new_categorical.copy()

In [123]:
case_01_numerical_cp = case_01_numerical_cp.astype('str')
case_01_new_numerical_cp = case_01_new_numerical_cp.astype('str')

### case_01_numerical

In [124]:
for i in range(100):
    for j in range(96):
        if "NaN" in case_01_numerical_cp.iloc[i][j] or case_01_numerical_cp.iloc[i][j]==np.nan:
            # case1Data.iloc[i][j] = '0.0'
            case_01_numerical_cp.loc[i,case_01_numerical_cp.columns[j]] = np.nan
case_01_numerical_cp

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,-2.590210,-1.943897,3.011972,-31.614019,-3.990017,3.692902,12.826477,-6.889479,3.443535,5.400120
1,-22.200026,-6.370690,7.792366,11.858663,NaN,-15.420530,21.683970,1.057192,2.559764,NaN,...,-3.573072,-1.583166,NaN,-28.880360,-6.552140,7.270303,10.039281,-7.678936,-0.189822,0.717526
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,NaN,2.484515,NaN,1.852533,...,-4.023667,2.648645,4.067629,-32.769482,-5.438118,5.889789,14.186614,-9.527944,0.400832,-2.458897
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,NaN,-0.442587,0.743039,-0.376102,...,-4.987491,3.992532,3.954274,-35.846878,-10.208559,7.024105,8.871258,-7.514400,-3.448892,-2.612379
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,NaN,19.057921,1.900578,6.133609,-0.771247,...,-4.021144,3.152862,3.366474,NaN,-3.632849,8.186575,10.532577,-7.509476,NaN,0.754905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,NaN,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,-2.473629,3.219622,7.296911,-31.600305,-1.008369,NaN,17.348757,-6.059881,4.196626,1.168894
96,-0.623049,-4.311540,7.208115,NaN,-17.232457,-11.173426,24.660276,NaN,8.513152,4.327020,...,1.541280,6.440714,4.503169,-29.506432,-6.276006,9.753088,17.345148,-3.650362,3.535945,NaN
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,NaN,...,-11.148246,NaN,-2.396112,-39.440232,-8.566806,-0.300405,7.651474,NaN,-4.384652,-1.665588
98,-25.058153,-7.338365,1.597154,12.151154,NaN,NaN,23.185303,3.751106,6.357028,5.208797,...,-6.120490,NaN,0.672467,-33.044920,-5.693242,6.465018,11.647115,-4.830073,-2.378979,-3.120303


In [125]:
mode_list_case_01 = case_01_numerical_cp.mode().iloc[0]

In [126]:
case_01_numerical_cp = case_01_numerical_cp.astype('str')
case_01_numerical_cp

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,-2.590210,-1.943897,3.011972,-31.614019,-3.990017,3.692902,12.826477,-6.889479,3.443535,5.400120
1,-22.200026,-6.370690,7.792366,11.858663,nan,-15.420530,21.683970,1.057192,2.559764,nan,...,-3.573072,-1.583166,nan,-28.880360,-6.552140,7.270303,10.039281,-7.678936,-0.189822,0.717526
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,nan,2.484515,nan,1.852533,...,-4.023667,2.648645,4.067629,-32.769482,-5.438118,5.889789,14.186614,-9.527944,0.400832,-2.458897
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,nan,-0.442587,0.743039,-0.376102,...,-4.987491,3.992532,3.954274,-35.846878,-10.208559,7.024105,8.871258,-7.514400,-3.448892,-2.612379
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,nan,19.057921,1.900578,6.133609,-0.771247,...,-4.021144,3.152862,3.366474,nan,-3.632849,8.186575,10.532577,-7.509476,nan,0.754905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,nan,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,-2.473629,3.219622,7.296911,-31.600305,-1.008369,nan,17.348757,-6.059881,4.196626,1.168894
96,-0.623049,-4.311540,7.208115,nan,-17.232457,-11.173426,24.660276,nan,8.513152,4.327020,...,1.541280,6.440714,4.503169,-29.506432,-6.276006,9.753088,17.345148,-3.650362,3.535945,nan
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,nan,...,-11.148246,nan,-2.396112,-39.440232,-8.566806,-0.300405,7.651474,nan,-4.384652,-1.665588
98,-25.058153,-7.338365,1.597154,12.151154,nan,nan,23.185303,3.751106,6.357028,5.208797,...,-6.120490,nan,0.672467,-33.044920,-5.693242,6.465018,11.647115,-4.830073,-2.378979,-3.120303


In [127]:
# DATASET_RAW['FACTORY_NUM'] = DATASET_RAW['FACTORY_NUM'].str.replace('공장', '라인')
for i in range(case_01_numerical_cp.shape[1]):
    case_01_numerical_cp[case_01_numerical_cp.columns[i]] = case_01_numerical_cp[case_01_numerical_cp.columns[i]].str.replace('nan',mode_list_case_01[i])
        
case_01_numerical_cp

,y,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.158895,-7.518161,5.423677,11.810360,-19.076321,-7.018273,23.841339,1.050206,3.132453,4.403462,...,-2.590210,-1.943897,3.011972,-31.614019,-3.990017,3.692902,12.826477,-6.889479,3.443535,5.400120
1,-22.200026,-6.370690,7.792366,11.858663,-10.143062,-15.420530,21.683970,1.057192,2.559764,-0.001413,...,-3.573072,-1.583166,-0.489808,-28.880360,-6.552140,7.270303,10.039281,-7.678936,-0.189822,0.717526
2,-19.357551,-4.229569,2.756115,12.446836,-17.567113,-12.984783,16.342276,2.484515,-1.356242,1.852533,...,-4.023667,2.648645,4.067629,-32.769482,-5.438118,5.889789,14.186614,-9.527944,0.400832,-2.458897
3,28.763803,-10.487072,1.460685,9.307533,-21.306282,-19.824912,16.342276,-0.442587,0.743039,-0.376102,...,-4.987491,3.992532,3.954274,-35.846878,-10.208559,7.024105,8.871258,-7.514400,-3.448892,-2.612379
4,43.779898,-13.066517,4.812794,10.641948,-17.691825,-10.397767,19.057921,1.900578,6.133609,-0.771247,...,-4.021144,3.152862,3.366474,-24.734686,-3.632849,8.186575,10.532577,-7.509476,-0.003119,0.754905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.113668,-3.746623,-0.265494,14.898749,-17.262103,-15.406944,25.744304,4.157661,6.127105,2.839280,...,-2.473629,3.219622,7.296911,-31.600305,-1.008369,-0.026005,17.348757,-6.059881,4.196626,1.168894
96,-0.623049,-4.311540,7.208115,10.171101,-17.232457,-11.173426,24.660276,-0.108392,8.513152,4.327020,...,1.541280,6.440714,4.503169,-29.506432,-6.276006,9.753088,17.345148,-3.650362,3.535945,-0.011575
97,102.295957,-15.662503,0.521416,7.628422,-18.867588,-20.632873,20.905762,-3.530738,0.965488,-0.001413,...,-11.148246,-0.005782,-2.396112,-39.440232,-8.566806,-0.300405,7.651474,-1.268347,-4.384652,-1.665588
98,-25.058153,-7.338365,1.597154,12.151154,-10.143062,-10.397767,23.185303,3.751106,6.357028,5.208797,...,-6.120490,-0.005782,0.672467,-33.044920,-5.693242,6.465018,11.647115,-4.830073,-2.378979,-3.120303


### case_01_new_numerical

In [128]:
# case_01_new_numerical_cp
for i in range(case_01_new_numerical_cp.shape[0]):
    for j in range(case_01_new_numerical_cp.shape[1]):
        if "NaN" in case_01_new_numerical_cp.iloc[i][j] or case_01_new_numerical_cp.iloc[i][j]==np.nan:
            # case1Data.iloc[i][j] = '0.0'
            case_01_new_numerical_cp.loc[i,case_01_new_numerical_cp.columns[j]] = np.nan
case_01_new_numerical_cp

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.4932,4.857913,12.334877,-17.075041,NaN,NaN,5.339252,NaN,-0.275014,NaN,...,-2.661989,3.802699,6.012018,-30.831163,NaN,6.983981,14.086256,NaN,3.391461,2.102585
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,-4.195056,2.549679,4.436252,NaN,-6.435776,5.799896,11.477398,-10.509565,-0.347968,-2.471949
2,-8.507726,1.639164,NaN,-19.167274,-15.745444,20.336708,NaN,NaN,2.756582,-6.619169,...,-3.809937,2.671304,3.119847,-33.272728,-3.679832,9.498865,NaN,-6.555100,-3.588808,-2.342992
3,-12.841493,NaN,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,-8.390266,0.930408,2.797268,-36.442482,-7.670354,8.428060,12.257391,-8.267102,-1.373772,-5.374916
4,-8.55516,NaN,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,NaN,...,-1.446679,NaN,7.196290,-31.796824,-7.212525,11.065901,8.004617,-10.170813,2.445537,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,NaN,3.956915,-1.673680,-5.303247,...,-6.531539,-1.911204,4.902708,-32.978883,-7.900873,8.286523,14.119678,-6.228438,-0.419140,-4.737494
996,-7.835061,5.410653,NaN,-18.169995,-15.071277,23.569444,2.922559,5.423952,NaN,-0.561963,...,-2.772379,3.177223,3.005404,-30.616184,-4.494307,8.773873,NaN,-8.891943,0.617816,NaN
997,-7.547603,NaN,NaN,NaN,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,-5.460523,0.778803,4.249185,-32.984152,-3.465181,NaN,13.666244,-9.850952,-2.478082,1.597624
998,-10.989069,3.416966,NaN,-17.205503,-10.092004,21.299033,2.201629,3.492949,NaN,-4.334677,...,-1.871098,NaN,3.734328,-33.292990,NaN,12.008307,12.634788,-5.447805,-0.529636,1.834458


In [129]:
mode_list_case_01_new = case_01_new_numerical_cp.mode().iloc[0]

In [130]:
mode_list_case_01_new

x_ 1             nan
 x_ 2      -0.006051
 x_ 3      10.008101
 x_ 4     -10.250633
 x_ 5     -10.005906
            ...     
 x_91      -0.489275
 x_92      10.006645
 x_93      -0.133124
 x_94      -0.011899
 x_95      -0.008240
Name: 0, Length: 95, dtype: object

In [131]:
case_01_new_numerical_cp = case_01_new_numerical_cp.astype('str')
case_01_new_numerical_cp

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.4932,4.857913,12.334877,-17.075041,nan,nan,5.339252,nan,-0.275014,nan,...,-2.661989,3.802699,6.012018,-30.831163,nan,6.983981,14.086256,nan,3.391461,2.102585
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,-4.195056,2.549679,4.436252,nan,-6.435776,5.799896,11.477398,-10.509565,-0.347968,-2.471949
2,-8.507726,1.639164,nan,-19.167274,-15.745444,20.336708,nan,nan,2.756582,-6.619169,...,-3.809937,2.671304,3.119847,-33.272728,-3.679832,9.498865,nan,-6.555100,-3.588808,-2.342992
3,-12.841493,nan,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,-8.390266,0.930408,2.797268,-36.442482,-7.670354,8.428060,12.257391,-8.267102,-1.373772,-5.374916
4,-8.55516,nan,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,nan,...,-1.446679,nan,7.196290,-31.796824,-7.212525,11.065901,8.004617,-10.170813,2.445537,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,nan,3.956915,-1.673680,-5.303247,...,-6.531539,-1.911204,4.902708,-32.978883,-7.900873,8.286523,14.119678,-6.228438,-0.419140,-4.737494
996,-7.835061,5.410653,nan,-18.169995,-15.071277,23.569444,2.922559,5.423952,nan,-0.561963,...,-2.772379,3.177223,3.005404,-30.616184,-4.494307,8.773873,nan,-8.891943,0.617816,nan
997,-7.547603,nan,nan,nan,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,-5.460523,0.778803,4.249185,-32.984152,-3.465181,nan,13.666244,-9.850952,-2.478082,1.597624
998,-10.989069,3.416966,nan,-17.205503,-10.092004,21.299033,2.201629,3.492949,nan,-4.334677,...,-1.871098,nan,3.734328,-33.292990,nan,12.008307,12.634788,-5.447805,-0.529636,1.834458


In [132]:
for i in range(0,case_01_new_numerical_cp.shape[1]):
    case_01_new_numerical_cp[case_01_new_numerical_cp.columns[i]] = case_01_new_numerical_cp[case_01_new_numerical_cp.columns[i]].str.replace('nan',mode_list_case_01_new[i])
   
case_01_new_numerical_cp

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.4932,4.857913,12.334877,-17.075041,-10.005906,14.545844,5.339252,-0.000574,-0.275014,-0.023505,...,-2.661989,3.802699,6.012018,-30.831163,-0.046115,6.983981,14.086256,-0.133124,3.391461,2.102585
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,-4.195056,2.549679,4.436252,-23.768014,-6.435776,5.799896,11.477398,-10.509565,-0.347968,-2.471949
2,-8.507726,1.639164,10.008101,-19.167274,-15.745444,20.336708,-0.015468,-0.000574,2.756582,-6.619169,...,-3.809937,2.671304,3.119847,-33.272728,-3.679832,9.498865,10.006645,-6.555100,-3.588808,-2.342992
3,-12.841493,-0.006051,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,-8.390266,0.930408,2.797268,-36.442482,-7.670354,8.428060,12.257391,-8.267102,-1.373772,-5.374916
4,-8.55516,-0.006051,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,-0.023505,...,-1.446679,-0.006351,7.196290,-31.796824,-7.212525,11.065901,8.004617,-10.170813,2.445537,-0.008240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.501994,7.364637,14.598509,-20.284036,-16.604685,20.682107,-0.015468,3.956915,-1.673680,-5.303247,...,-6.531539,-1.911204,4.902708,-32.978883,-7.900873,8.286523,14.119678,-6.228438,-0.419140,-4.737494
996,-7.835061,5.410653,10.008101,-18.169995,-15.071277,23.569444,2.922559,5.423952,-0.019162,-0.561963,...,-2.772379,3.177223,3.005404,-30.616184,-4.494307,8.773873,10.006645,-8.891943,0.617816,-0.008240
997,-7.547603,-0.006051,10.008101,-10.250633,-14.997699,22.129570,4.148231,1.814772,1.798028,-4.611600,...,-5.460523,0.778803,4.249185,-32.984152,-3.465181,-0.489275,13.666244,-9.850952,-2.478082,1.597624
998,-10.989069,3.416966,10.008101,-17.205503,-10.092004,21.299033,2.201629,3.492949,-0.019162,-4.334677,...,-1.871098,-0.006351,3.734328,-33.292990,-0.046115,12.008307,12.634788,-5.447805,-0.529636,1.834458


In [133]:
case_01_new_numerical_cp.head(12)

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_86,x_87,x_88,x_89,x_90,x_91,x_92,x_93,x_94,x_95
0,-6.4932,4.857913,12.334877,-17.075041,-10.005906,14.545844,5.339252,-0.000574,-0.275014,-0.023505,...,-2.661989,3.802699,6.012018,-30.831163,-0.046115,6.983981,14.086256,-0.133124,3.391461,2.102585
1,-6.688701,6.402298,13.321841,-14.890575,-15.208925,22.978112,-3.424436,2.773030,1.326065,-2.664610,...,-4.195056,2.549679,4.436252,-23.768014,-6.435776,5.799896,11.477398,-10.509565,-0.347968,-2.471949
2,-8.507726,1.639164,10.008101,-19.167274,-15.745444,20.336708,-0.015468,-0.000574,2.756582,-6.619169,...,-3.809937,2.671304,3.119847,-33.272728,-3.679832,9.498865,10.006645,-6.555100,-3.588808,-2.342992
3,-12.841493,-0.006051,11.307615,-22.689726,-17.290155,20.984080,-1.775278,2.554297,0.315108,-4.498758,...,-8.390266,0.930408,2.797268,-36.442482,-7.670354,8.428060,12.257391,-8.267102,-1.373772,-5.374916
4,-8.55516,-0.006051,13.122784,-12.977789,-14.144455,24.086075,6.270027,2.181681,6.731993,-0.023505,...,-1.446679,-0.006351,7.196290,-31.796824,-7.212525,11.065901,8.004617,-10.170813,2.445537,-0.008240
5,-11.581414,2.033466,8.638852,-12.249232,-19.351819,23.017498,-0.297364,-0.000574,-2.260586,-6.728059,...,-7.645013,0.762545,4.638621,-23.768014,-4.948007,6.342297,10.006645,-7.266876,-4.206440,1.928793
6,-4.981761,8.519576,15.443969,-17.160366,-10.005906,21.496286,-1.069276,1.243661,5.235561,-3.426256,...,-0.001968,3.230915,5.279225,-32.587318,-1.449168,12.116432,14.425660,-0.133124,-0.011899,-2.228835
7,-10.01624,2.600921,10.008101,-10.250633,-13.503943,23.122776,4.020541,3.756771,-1.300190,-7.363724,...,-6.878087,-2.796069,2.992652,-36.673486,-6.466256,4.662557,10.006645,-6.408185,-5.379945,-2.051513
8,-6.125908,8.055846,15.459509,-15.913496,-10.351638,29.035828,9.635380,11.647153,6.253733,-2.514735,...,-1.300484,5.466414,5.333344,-23.768014,-2.617554,9.113186,10.006645,-10.890435,2.126627,-0.008240
9,-4.593261,5.589658,10.008101,-15.195908,-12.087648,24.170861,9.146394,9.290994,2.304327,-2.953640,...,-0.855445,0.831662,4.399209,-31.382303,-1.791063,7.650967,13.604060,-4.567645,5.912064,6.320480


### case_01_categorical

In [134]:
# case_01_categorical_cp

In [135]:
for i in range(case_01_categorical_cp.shape[0]):
    for j in range(case_01_categorical_cp.shape[1]):
        if "NaN" in case_01_categorical_cp.iloc[i][j] or case_01_categorical_cp.iloc[i][j]==np.nan:
            # case1Data.iloc[i][j] = '0.0'
            case_01_categorical_cp.loc[i,case_01_categorical_cp.columns[j]] = np.nan
case_01_categorical_cp

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,I,NaN,H,J,K
1,H,NaN,H,I,I
2,NaN,I,K,J,G
3,K,H,I,I,NaN
4,H,H,K,J,J
...,...,...,...,...,...
95,H,I,G,NaN,H
96,H,I,I,J,K
97,G,H,NaN,G,K
98,G,I,I,H,NaN


In [136]:
mode_list_case_01_cat = case_01_categorical_cp.mode().iloc[0]

In [137]:
case_01_categorical_cp = case_01_categorical_cp.astype('str')
case_01_categorical_cp

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,I,nan,H,J,K
1,H,nan,H,I,I
2,nan,I,K,J,G
3,K,H,I,I,nan
4,H,H,K,J,J
...,...,...,...,...,...
95,H,I,G,nan,H
96,H,I,I,J,K
97,G,H,nan,G,K
98,G,I,I,H,nan


In [138]:
for i in range(case_01_categorical_cp.shape[1]):
    case_01_categorical_cp[case_01_categorical_cp.columns[i]] = case_01_categorical_cp[case_01_categorical_cp.columns[i]].str.replace('nan',mode_list_case_01_cat[i])
        
case_01_categorical_cp

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,I,I,H,J,K
1,H,I,H,I,I
2,G,I,K,J,G
3,K,H,I,I,H
4,H,H,K,J,J
...,...,...,...,...,...
95,H,I,G,I,H
96,H,I,I,J,K
97,G,H,I,G,K
98,G,I,I,H,H


### case_01_categorical_new

In [139]:
# case_01_new_categorical_cp

In [140]:
for i in range(case_01_new_categorical_cp.shape[0]):
    for j in range(case_01_new_categorical_cp.shape[1]):
        if "NaN" in case_01_new_categorical_cp.iloc[i][j] or case_01_new_categorical_cp.iloc[i][j]==np.nan:
            # case1Data.iloc[i][j] = '0.0'
            case_01_new_categorical_cp.loc[i,case_01_new_categorical_cp.columns[j]] = np.nan
case_01_new_categorical_cp

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,G,I,G,J,NaN
1,I,I,NaN,NaN,H
2,NaN,H,NaN,H,K
3,G,H,K,K,J
4,H,I,J,H,G
...,...,...,...,...,...
995,NaN,I,H,I,NaN
996,J,I,NaN,G,G
997,H,H,H,H,J
998,H,I,G,G,G


In [141]:
mode_list_case_01_cat_new = case_01_new_categorical_cp.mode().iloc[0]

In [142]:
case_01_new_categorical_cp = case_01_new_categorical_cp.astype('str')
case_01_new_categorical_cp

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,G,I,G,J,nan
1,I,I,nan,nan,H
2,nan,H,nan,H,K
3,G,H,K,K,J
4,H,I,J,H,G
...,...,...,...,...,...
995,nan,I,H,I,nan
996,J,I,nan,G,G
997,H,H,H,H,J
998,H,I,G,G,G


In [143]:
for i in range(case_01_new_categorical_cp.shape[1]):
    case_01_new_categorical_cp[case_01_new_categorical_cp.columns[i]] = case_01_new_categorical_cp[case_01_new_categorical_cp.columns[i]].str.replace('nan',mode_list_case_01_cat_new[i])
        
case_01_new_categorical_cp

,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,G,I,G,J,G
1,I,I,J,K,H
2,H,H,J,H,K
3,G,H,K,K,J
4,H,I,J,H,G
...,...,...,...,...,...
995,H,I,H,I,G
996,J,I,J,G,G
997,H,H,H,H,J
998,H,I,G,G,G


## Merging dataset

In [32]:
# case_01_numerical_cp
# case_01_categorical_cp

# case_01_new_numerical_cp
# case_01_new_categorical_cp 

In [32]:
Y = case_01_numerical_cp[['y']]
X_01 = case_01_numerical_cp[case_01_numerical_cp.columns[1:]]
X_02 = case_01_new_numerical_cp
X_cat_01 = case_01_categorical_cp
X_cat_02 = case_01_new_categorical_cp

# X_01

In [38]:
X_01.to_csv('../dataset/x_numerical.csv')
X_02.to_csv('../dataset/x_new_numerical.csv')
X_cat_01.to_csv('../dataset/x_cat.csv')
X_cat_02.to_csv('../dataset/x_new_cat.csv')
Y.to_csv('../dataset/y.csv')